In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import enchant 
import nltk
from nltk.corpus import words
import re
dict_en = enchant.Dict("en_US")

In [2]:
job_title_need_clean = pd.read_csv("data/test/jobTitle_employment.csv")
job_title_need_clean = job_title_need_clean[['c_employment_title']]

job_title_need_clean = job_title_need_clean.apply(lambda x: x.str.strip()).replace(np.nan, '')

job_title_need_clean['suggest_title'] = ''
job_title_need_clean['suggest_title_score'] = 0



# job_title_need_clean['suggest_title'] = job_title_need_clean.c_employment_title
# job_title_need_clean['suggest_title_score'] = 1000 # the flag show that this title correct, don't need to clean

# Step 1: Check all words in job title are correctly spelling

In [3]:
total = len(job_title_need_clean)
for index, row in job_title_need_clean.iterrows():
    if index % 1000 == 0:
        print(index, '/', total)
    
    job_title = row['c_employment_title']
    job_title = ''.join(e for e in row['c_employment_title'] if e.isalnum() or e == ' ')
    
    is_correct = True
    for word in job_title.split():
        if not dict_en.check(word):
            is_correct = False
            break
    if is_correct:
        # All words in job_title are correctly spelling
        job_title_need_clean.set_value(index, 'suggest_title', job_title)
        job_title_need_clean.set_value(index, 'suggest_title_score', 1000) 
    

0 / 200000
1000 / 200000
2000 / 200000
3000 / 200000
4000 / 200000
5000 / 200000
6000 / 200000
7000 / 200000
8000 / 200000
9000 / 200000
10000 / 200000
11000 / 200000
12000 / 200000
13000 / 200000
14000 / 200000
15000 / 200000
16000 / 200000
17000 / 200000
18000 / 200000
19000 / 200000
20000 / 200000
21000 / 200000
22000 / 200000
23000 / 200000
24000 / 200000
25000 / 200000
26000 / 200000
27000 / 200000
28000 / 200000
29000 / 200000
30000 / 200000
31000 / 200000
32000 / 200000
33000 / 200000
34000 / 200000
35000 / 200000
36000 / 200000
37000 / 200000
38000 / 200000
39000 / 200000
40000 / 200000
41000 / 200000
42000 / 200000
43000 / 200000
44000 / 200000
45000 / 200000
46000 / 200000
47000 / 200000
48000 / 200000
49000 / 200000
50000 / 200000
51000 / 200000
52000 / 200000
53000 / 200000
54000 / 200000
55000 / 200000
56000 / 200000
57000 / 200000
58000 / 200000
59000 / 200000
60000 / 200000
61000 / 200000
62000 / 200000
63000 / 200000
64000 / 200000
65000 / 200000
66000 / 200000
67000 / 

In [4]:
len(job_title_need_clean[job_title_need_clean.suggest_title_score == 1000])

159530

In [5]:
one_gram = pd.read_csv("one_gram.csv", na_values = 'NULL', keep_default_na= False)
columns = ['word', 'frequency', 'suggest_word', 'score']
words_not_in_dict_en = pd.DataFrame(columns=columns)


In [7]:
i = 0
for index, row in one_gram.iterrows():
    if index % 100 == 0:
        print(index, '/', total)
        
    word = row['word']
    if not dict_en.check(word):
        frequency = row['frequency']
        words_not_in_dict_en.set_value(i, 'word', word)
        words_not_in_dict_en.set_value(i, 'frequency', frequency)
        
        suggest_words = dict_en.suggest(word)
        if len(suggest_words) > 0 :
            suggest_word = suggest_words[0]
            score = fuzz.ratio(word, suggest_word)
            
            words_not_in_dict_en.set_value(i, 'suggest_word', suggest_word)
            words_not_in_dict_en.set_value(i, 'score', score )
        else:
            words_not_in_dict_en.set_value(i, 'suggest_word', '')
            words_not_in_dict_en.set_value(i, 'score', 0)
        i = i + 1
words_not_in_dict_en.to_csv("words_not_in_dict_en.csv")

0 / 200000
100 / 200000
200 / 200000
300 / 200000
400 / 200000
500 / 200000
600 / 200000
700 / 200000
800 / 200000
900 / 200000
1000 / 200000
1100 / 200000
1200 / 200000
1300 / 200000
1400 / 200000
1500 / 200000
1600 / 200000
1700 / 200000
1800 / 200000
1900 / 200000
2000 / 200000
2100 / 200000
2200 / 200000
2300 / 200000
2400 / 200000
2500 / 200000
2600 / 200000
2700 / 200000
2800 / 200000
2900 / 200000
3000 / 200000
3100 / 200000
3200 / 200000
3300 / 200000
3400 / 200000
3500 / 200000
3600 / 200000
3700 / 200000
3800 / 200000
3900 / 200000
4000 / 200000
4100 / 200000
4200 / 200000
4300 / 200000
4400 / 200000
4500 / 200000
4600 / 200000
4700 / 200000
4800 / 200000
4900 / 200000
5000 / 200000
5100 / 200000
5200 / 200000
5300 / 200000
5400 / 200000
5500 / 200000
5600 / 200000
5700 / 200000
5800 / 200000
5900 / 200000
6000 / 200000
6100 / 200000
6200 / 200000
6300 / 200000
6400 / 200000
6500 / 200000
6600 / 200000
6700 / 200000
6800 / 200000
6900 / 200000
7000 / 200000
7100 / 200000
7200

In [8]:
words_not_in_dict_en_score_ge_95 = words_not_in_dict_en[words_not_in_dict_en.score >= 95]

In [9]:
correct_word_dict = {}

nb_of_fixed_title = 0
total = len(words_not_in_dict_en_score_ge_95)
for index, row in words_not_in_dict_en_score_ge_95.iterrows():  
    if index % 100 == 0:
        print(index, '/', total)
    
    wrong_word = row['word']    
    job_titles_spelling_wrong = \
        job_title_need_clean[job_title_need_clean.c_employment_title.str.contains(wrong_word, na=False)]    
    
    if len(job_titles_spelling_wrong) > 0:
        for index_2, row_2 in job_titles_spelling_wrong.iterrows():            
            original_title = row_2['c_employment_title']     
            # Find exactly the wrong word in job title. 
            if len(re.findall('\\b'+wrong_word+'\\b', original_title, flags=re.IGNORECASE)) > 0:
                suggest_word = row['suggest_word']
                
                correct_word_dict[suggest_word] = wrong_word
                
                suggest_title = original_title.replace(wrong_word, suggest_word)
                score = fuzz.ratio(original_title,suggest_title)
                job_title_need_clean.set_value(index_2, 'suggest_title', suggest_title)
                job_title_need_clean.set_value(index_2, 'suggest_title_score', score)     
                
                if nb_of_fixed_title % 100 == 0:
                    print("nb_of_fixed_title: ", nb_of_fixed_title)
                    
                nb_of_fixed_title = nb_of_fixed_title + 1
                
job_title_need_clean.to_csv("job_title_need_clean.csv")

nb_of_fixed_title:  0
nb_of_fixed_title:  100
nb_of_fixed_title:  200
nb_of_fixed_title:  300
nb_of_fixed_title:  400
nb_of_fixed_title:  500
nb_of_fixed_title:  600
nb_of_fixed_title:  700
nb_of_fixed_title:  800
nb_of_fixed_title:  900
nb_of_fixed_title:  1000
nb_of_fixed_title:  1100
nb_of_fixed_title:  1200
nb_of_fixed_title:  1300
nb_of_fixed_title:  1400
nb_of_fixed_title:  1500
100 / 4918
nb_of_fixed_title:  1600
nb_of_fixed_title:  1700
nb_of_fixed_title:  1800
nb_of_fixed_title:  1900
nb_of_fixed_title:  2000
200 / 4918
nb_of_fixed_title:  2100
nb_of_fixed_title:  2200
nb_of_fixed_title:  2300
nb_of_fixed_title:  2400
nb_of_fixed_title:  2500
nb_of_fixed_title:  2600
nb_of_fixed_title:  2700
nb_of_fixed_title:  2800
nb_of_fixed_title:  2900
nb_of_fixed_title:  3000
1100 / 4918
1200 / 4918
nb_of_fixed_title:  3100
nb_of_fixed_title:  3200
nb_of_fixed_title:  3300
1900 / 4918
2100 / 4918
nb_of_fixed_title:  3400
2200 / 4918
2400 / 4918
nb_of_fixed_title:  3500
2600 / 4918
2700 /

In [12]:
len(job_title_need_clean[(job_title_need_clean.suggest_title_score >= 95) & \
                    (job_title_need_clean.suggest_title_score < 1000)])

4528

In [15]:
correct_word_dict = sorted(correct_word_dict.items(), key = lambda x: x[1], reverse=True)
pd.DataFrame(correct_word_dict).to_csv('correct_word_dict.csv')

In [18]:
len(job_title_need_clean[(job_title_need_clean.suggest_title_score < 95)])

35942

In [26]:
words_not_in_dict_en[(words_not_in_dict_en.frequency >= 50) & (words_not_in_dict_en.score < 95)].to_csv("frequenly_words_not_in_dict_en.csv")

#### Find manually nearest word

In [35]:
frequenly_words_correct_manual = pd.read_csv("frequenly_words_not_in_dict_en.csv")

In [36]:
frequenly_words_correct_manual.head()

,Unnamed: 0,word,frequency,suggest_word,score,Unnamed: 5
0,0,EMEA,795,EMEA,200,NaN
1,1,Dba,583,Database administrators,200,NaN
2,2,CRM,558,Customer relationship management,200,NaN
3,3,UI,532,User interface,200,NaN
4,4,Online,526,Online,200,NaN


In [38]:
correct_word_dict = {}

nb_of_fixed_title = 0
total = len(frequenly_words_correct_manual)

for index, row in frequenly_words_correct_manual.iterrows():  
    wrong_word = row['word']    
    print("Word: ", wrong_word)
    
    job_titles_spelling_wrong = \
        job_title_need_clean[job_title_need_clean.c_employment_title.str.contains(wrong_word, na=False)]    
    
    if len(job_titles_spelling_wrong) > 0:
        for index_2, row_2 in job_titles_spelling_wrong.iterrows():            
            original_title = row_2['c_employment_title']     
            
            # Find exactly the wrong word in job title. 
            if len(re.findall('\\b'+wrong_word+'\\b', \
                              original_title, flags=re.IGNORECASE)) > 0:
                
                suggest_word = row['suggest_word']
                correct_word_dict[suggest_word] = wrong_word
                suggest_title = original_title.replace(wrong_word, suggest_word)
                
                print("Original: ", original_title)
                print("suggest_title: ", suggest_title)
                
                score = fuzz.ratio(original_title,suggest_title)
                print("index 2: ", index_2)
                job_title_need_clean.set_value(index_2, 'suggest_title', suggest_title)
                job_title_need_clean.set_value(index_2, 'suggest_title_score', score)     
                
                if nb_of_fixed_title % 100 == 0:
                    print("nb_of_fixed_title: ", nb_of_fixed_title)
                    
                nb_of_fixed_title = nb_of_fixed_title + 1
                
job_title_need_clean.to_csv("job_title_need_clean.csv")

Word:  EMEA
Original:  Project Manager - EMEA Sales Operations
suggest_title:  Project Manager - EMEA Sales Operations
index 2:  79
nb_of_fixed_title:  0
Original:  SIEM Consultant EMEA
suggest_title:  SIEM Consultant EMEA
index 2:  3859
Original:  Commercial Data Manager EMEA
suggest_title:  Commercial Data Manager EMEA
index 2:  7586
Original:  2VP Monitoring &amp; Testing EMEA
suggest_title:  2VP Monitoring &amp; Testing EMEA
index 2:  7754
Original:  Technology Consultant, EMEA International Expertise Team
suggest_title:  Technology Consultant, EMEA International Expertise Team
index 2:  7863
Original:  Technology Consultant, EMEA Technical Consulting and Services
suggest_title:  Technology Consultant, EMEA Technical Consulting and Services
index 2:  7864
Original:  Director Sales Engineering EMEA
suggest_title:  Director Sales Engineering EMEA
index 2:  9021
Original:  ALM &amp; BSM Solution Consultant, EMEA MSE Channel &amp; Alliance Pre-Sales, HP Software
suggest_title:  ALM &am

In [43]:
import numpy as np
test_row_indexs = np.random.randint(0, high=200000, size= 1000)

In [63]:
# test_rows = pd.DataFrame(columns=columns)
# for i in range(0, len(test_row_indexs)):
#job_title_need_clean.loc[[test_row_indexs]]
for index, row in job_title_need_clean.iterrows():
    if index in test_row_indexs:
        print(row['c_employment_title'], ',', \
              row['suggest_title'], ',',\
              row['suggest_title_score'])


Project Manager - EMEA Sales Operations , Project Manager - EMEA Sales Operations , 100
Web Designer , Web Designer , 1000
Graphic Artist and Web Designer , Graphic Artist and Web Designer , 1000
CIO , CIO , 100
Systems Engineer , Systems Engineer , 1000
Role &ndash; Sr. Hyperion Lead &amp; Admin , Role &ndash; Sr. Hyperion Lead &amp; administrator , 89
Sales Development Manager/SE Manager, Asia Pacific Region ,  , 0
Vice President Storage &amp; Trading , Vice President Storage amp Trading , 1000
IT Systems Engineer , IT Systems Engineer , 1000
Drilling Engineer , Drilling Engineer , 1000
Consultant Drilling Fluids and Cement Supervisor , Consultant Drilling Fluids and Cement Supervisor , 1000
Senior Sales Representative , Senior Sales Representative , 1000
Web Production Manager , Web Production Manager , 1000
Project Planner , Project Planner , 1000
Developer , Developer , 1000
Platform Engineer , Platform Engineer , 1000
SAP BI Architect , SAP BI Architect , 1000
Manager , Manager ,

In [69]:
evaluation = pd.read_csv("evalutaion_job_title.csv")
print(len(evaluation[evaluation.check == 'y']))
print(len(evaluation[evaluation.check == 'n']))

838
162
